# Initial calculation code

In [1]:
import numpy as np

def geom_avg(vals):
    """
    Compute the geometric average of a list of values.
    
    The values need not be a list, but simply anything with a len() and []
    """
    rval=1.0
    count = 0
    for val in vals:
        val = vals[count]
        if val != 0:
            rval *= val
            count+=1
    if count != 0:
        rval = pow(rval, 1.0/count)
    return(rval)

def geom_avg_mat(mat, coeffs = None):
    '''
    Computes the geometric average of the columns of a matrix.  Returns
    an np.array of dimension [nRowsOfMat], i.e. a vector.  
    
    :param mat: Must be an numpy.array of shape [nRows, nCols]
    :param coeffs:  If not None, it is a list like object with nColsOfMat elements.
    We multiply column 0 of mat by coeffs[0], column 1 of mat by coeffs[1], etc
    and then do the geometric average of the columns.  Essentially this weights the
    columns.
    '''
    """
    """
    size = mat.shape[0]
    rval = np.ones([size])
    for row in range(size):
        if np.any(coeffs):
            theRow = mat[row,:] * np.array(coeffs)
        else:
            theRow = mat[row,:]
        rval[row] = geom_avg(theRow)
    return(rval)

def bpriorities(mat, error = 1e-10):
    """
    Calculates priorities using Bill's method
    """
    size = mat.shape[0]
    vec = np.ones([size])
    diff = 1
    count=0
    while diff >= error and count < 100:
        nextv = geom_avg_mat(mat, vec)
        #nextv = nextv/max(nextv)
        diff = max(abs(nextv - vec))
        vec = nextv
        count+=1
    return(vec/sum(vec))

def gm_priorities(mat):
    '''
    Calculates the priorities using the geometric mean method
    :param mat: An numpy.array of dimension [size,size]
    '''
    rval = geom_avg_mat(mat)
    rval = rval / sum(rval)
    return(rval)

def harker_fix(mat):
    """
    Performs Harkers fix on the numpy matrix mat.  It returns a copy with the fix.
    The function does not change the matrix mat.
    :param mat:
    :return:
    """
    nrows = mat.shape[0]
    ncols = mat.shape[1]
    rval = mat.copy()
    for row in range(nrows):
        val = 1
        for col in range(ncols):
            if col != row and mat[row,col]==0:
                val+=1
        rval[row,row]=val
    return(rval)

def largest_eigen(mat, error = 1e-10, use_harker = False):
    if use_harker:
        mat = harker_fix(mat)
    size = mat.shape[0]
    vec = np.ones([size])
    diff = 1
    while diff > error:
        nextv = np.matmul(mat, vec)
        nextv = nextv/sum(nextv)
        diff = max(abs(nextv - vec))
        vec = nextv
    return(vec)

def priority_error(pwmat, privec):
    rval = 0
    diffsum = 0
    count = 0
    size = pwmat.shape[0]
    for i in range(0, size):
        for j in range (0, size):
            if pwmat[i,j] != 0:
                diffsum += (pwmat[i,j] - privec[i]/privec[j])**2
                count += 1
    if count == 0:
        return 0
    else:
        return diffsum ** (1.0/2)
    
def ratio_priority_error(pwmat, privec):
    rval = 1
    diffsum = 0
    count = 0
    ratio = 1
    score = 0
    size = pwmat.shape[0]
    for i in range(0, size):
        for j in range (0, size):
            if (pwmat[i,j] >= 1) and (i != j):
                ratio = pwmat[i,j]/(privec[i]/privec[j])
                if ratio >= 1:
                    score = ratio - 1
                else:
                    score = 1/ratio - 1
                diffsum += score
                count += 1
                #print("ratio={} diffprod={}".format(ratio, diffprod))
    if count == 0:
        return 0
    else:
        return diffsum * (1.0/count)
    
def ratio_priority_error_prod(pwmat, privec):
    rval = 1
    diffprod = 1
    count = 0
    ratio = 1
    size = pwmat.shape[0]
    for i in range(0, size):
        for j in range (0, size):
            if (pwmat[i,j] >= 1) and (i != j):
                ratio = pwmat[i,j]/(privec[i]/privec[j])
                diffprod *= ratio
                count += 1
                print("ratio={} diffprod={}".format(ratio, diffprod))
    print(diffprod)
    if count == 0:
        return 0
    else:
        return diffprod ** (1.0/count)
    
def ratio_mat(pv):
    size = len(pv)
    rval = np.identity(n=size)
    for row in range(size):
        for col in range(size):
            rval[row,col]=pv[row]/pv[col]
    return rval

# First round of calculations

## The matrix and largest eigen

In [2]:
mat = np.array([[1, 2, 1/9], [1/2, 1, 2], [9, 1/2, 1]])
mat

array([[ 1.        ,  2.        ,  0.11111111],
       [ 0.5       ,  1.        ,  2.        ],
       [ 9.        ,  0.5       ,  1.        ]])

In [3]:
leigen = largest_eigen(mat, error=1e-15)
leigen

array([ 0.18598961,  0.30706208,  0.50694832])

## Calculate the mismatch of the priority vector vs the pairwise matrix

In [4]:
priority_error(mat, leigen)

6.7801209182673734

## Check gradient of error function
It should be all zeroes at `leigen` if that is a minimimum.

In [5]:
my_error = lambda x: priority_error(mat, x)

In [6]:
from scipy.misc import derivative
from scipy.optimize import approx_fprime, minimize

In [7]:
eps = 1e-14
approx_fprime(leigen, my_error, eps)

array([ 11.36868377,   0.        ,  -4.26325641])

## Minimize priority_error(mat, *)
**It is not all zeroes, it looks like we can get a smaller error!**

In [8]:
rval = minimize(my_error, leigen, bounds=[(.01, 1), (.01, 1), (.01, 1)])

In [9]:
newpv = rval.x / sum(rval.x)
newpv

array([ 0.08367377,  0.23820359,  0.67812264])

In [10]:
# This is not the same, let's see them both together
leigen, newpv

(array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.08367377,  0.23820359,  0.67812264]))

In [11]:
# Let's look at the error of the eigen vs newpv
my_error(leigen), my_error(newpv)

(6.7801209182673734, 4.1537975084868064)

In [12]:
# Well that is surprising, approximately a 10% decline in error
# Let's consider the ratio matrix for both eigen and newpv
ratio_mat(leigen), None, ratio_mat(newpv)

(array([[ 1.        ,  0.60570686,  0.36688081],
        [ 1.65096362,  1.        ,  0.60570686],
        [ 2.72568089,  1.65096362,  1.        ]]),
 None,
 array([[ 1.        ,  0.35127   ,  0.12339033],
        [ 2.84681298,  1.        ,  0.35126919],
        [ 8.10436291,  2.84681957,  1.        ]]))

In [13]:
ratio_mat(leigen)

array([[ 1.        ,  0.60570686,  0.36688081],
       [ 1.65096362,  1.        ,  0.60570686],
       [ 2.72568089,  1.65096362,  1.        ]])

In [14]:
my_error(rval.x)

4.1537975084868064

## Try doing a similar calculation with my new priority calculation

In [15]:
bpv=bpriorities(mat)

In [16]:
leigen, bpv, newpv

(array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.08367377,  0.23820359,  0.67812264]))

Whoops, I forgot my new calculation agrees with the standard eigen for 3x3 matrices, let's just check for the transpose as well, for giggles

In [17]:
tbpv = bpriorities(mat.transpose())
ibpv = 1/bpv
ibpv = ibpv / sum(ibpv)
tbpv, ibpv

(array([ 0.50694832,  0.30706208,  0.18598961]),
 array([ 0.50694832,  0.30706208,  0.18598961]))

In [18]:
largest_eigen(mat.transpose())

array([ 0.50694832,  0.30706208,  0.18598961])

## Trying a new idea for convergence

In [19]:
def bill_iter(mat, p):
    rval = p/sum(p)
    size = len(p)
    for i in range(size):
        c = 1
        for j in range(size):
            if mat[i,j] >= 1:
                err = mat[i,j]/(p[i]/p[j])
            else:
                err = mat[j,i]/(p[j]/p[i])
            c *= err
        c = c ** (1/(2*(size-1)))
        rval[i]*=c
    rval=rval/sum(rval)
    return rval

In [20]:
p1=bill_iter(mat, newpv)
show_vals = [p1]
for i in range(50):
    show_vals.append(bill_iter(mat, show_vals[-1]))
show_vals

[array([ 0.07469302,  0.31996727,  0.60533971]),
 array([ 0.07447814,  0.43346536,  0.4920565 ]),
 array([ 0.08331014,  0.5509117 ,  0.36577816]),
 array([ 0.1063368 ,  0.63085349,  0.26280971]),
 array([ 0.15685457,  0.64358611,  0.19955931]),
 array([ 0.25643088,  0.57251478,  0.17105434]),
 array([ 0.41191741,  0.42189191,  0.16619069]),
 array([ 0.56762398,  0.25202821,  0.18034781]),
 array([ 0.63755448,  0.13420867,  0.22823685]),
 array([ 0.57763266,  0.07374461,  0.34862272]),
 array([ 0.38421096,  0.0437701 ,  0.57201894]),
 array([ 0.16565286,  0.02721272,  0.80713443]),
 array([ 0.05194343,  0.02031331,  0.92774326]),
 array([ 0.01635334,  0.02346603,  0.96018063]),
 array([ 0.00696487,  0.04804052,  0.94499461]),
 array([ 0.00473132,  0.16104915,  0.83421954]),
 array([ 0.00417358,  0.53621238,  0.45961404]),
 array([ 0.0032528 ,  0.89437098,  0.10237622]),
 array([ 0.00376535,  0.97930676,  0.01692789]),
 array([ 0.01045061,  0.98530729,  0.0042421 ]),
 array([ 0.07008281,

Okay that was **STUPID**

## Trying ratio_priority_error instead

In [21]:
ratio_priority_error(mat, leigen), ratio_priority_error(mat, newpv)

(2.3019272488946267, 3.1659260204051973)

Okay, so the newpv is not as good on this measurement, let's try to minimize again

In [22]:
my_error_ratio = lambda x: ratio_priority_error(mat, x)

In [23]:
rval_ratio = minimize(my_error_ratio, [10,1,1], bounds=[(.01, 1), (.01, 1), (.01, 1)])

In [24]:
ratio_priority_error(mat, rval_ratio.x)

2.301927248894641

In [25]:
rval_ratio.x/sum(rval_ratio.x)

array([ 0.18598962,  0.30706205,  0.50694833])

In [26]:
leigen

array([ 0.18598961,  0.30706208,  0.50694832])

It seems like the ratio priority error minimizing vector is leigen in this case.  This is weird, but I think it might be an artifact of the largest eigen having the doppelganger property for 3x3 matrices.

Just for giggles, let's verify for this case that doppelganger error is the same as original

In [27]:
ratio_priority_error(mat, leigen), ratio_priority_error(mat.transpose(), 1/leigen)

(2.3019272488946267, 2.3019272488946263)

## Trying ratio priority error again, with a wildly inconsistent 4x4 instead

In [28]:
mat4 = np.array([
    [1, 8, 1/10, 1/10],
    [1/8, 1, 4, 1/10],
    [10 , 1/4, 1, 5],
    [10, 10, 1/5, 1]
])

In [29]:
leigen4 = largest_eigen(mat4)

In [30]:
my_error_ratio4 = lambda x: ratio_priority_error(mat4, x)

In [31]:
rval_ratio = minimize(my_error_ratio4, leigen4, bounds=[(.001, 1), (.001, 1), (.001, 1), (.001, 1)])

In [32]:
newpv4 = rval_ratio.x/sum(rval_ratio.x)

In [33]:
leigen4, newpv4

(array([ 0.14684422,  0.15937263,  0.35162132,  0.34216183]),
 array([ 0.13517903,  0.12521498,  0.32696025,  0.41264574]))

In [34]:
ratio_priority_error(mat4, leigen4), ratio_priority_error(mat4, newpv4)

(4.9164739761337177, 4.7683649755766364)

Okay, we have an example where largest eigen disagrees with the new method.  Notice that they have different rankings of the alternatives as well

* largest_eigen: 4, 3, 1, 2
* new_idea     : 3, 4, 2, 1

In [35]:
bpv4 = bpriorities(mat4)
leigen4, newpv4, bpv4

(array([ 0.14684422,  0.15937263,  0.35162132,  0.34216183]),
 array([ 0.13517903,  0.12521498,  0.32696025,  0.41264574]),
 array([ 0.10637135,  0.09457899,  0.37607953,  0.42297012]))

Notice that the `bpriorities()` are different than either the eigen or new method